# PyDebFlow Demonstration Notebook

This notebook demonstrates how to use the **PyDebFlow** library for debris flow, avalanche, and lahar simulations.

## Installation

```bash
# Install from PyPI
pip install pydebflow

# With visualization support
pip install pydebflow[visualization]

# Full installation (GUI, visualization, raster I/O)
pip install pydebflow[all]
```

In [ ]:
# For development, add src to path
import sys
sys.path.insert(0, '..')

# Core imports
import numpy as np
import matplotlib.pyplot as plt

# PyDebFlow imports
import src as pydebflow

print(f"PyDebFlow version: {pydebflow.__version__}")

## 1. Creating Terrain

PyDebFlow can work with synthetic terrain or load real DEM files.

In [ ]:
# Create synthetic terrain (25° slope with a channel)
terrain = pydebflow.Terrain.create_synthetic_slope(
    rows=80,
    cols=60,
    cell_size=10.0,  # meters
    slope_angle=25.0,
    add_channel=True
)

print(f"Terrain dimensions: {terrain.rows} x {terrain.cols} cells")
print(f"Cell size: {terrain.cell_size} m")
print(f"Elevation range: {terrain.elevation.min():.1f} to {terrain.elevation.max():.1f} m")

In [ ]:
# Visualize the terrain
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(terrain.elevation, cmap='terrain', origin='upper')
ax.set_title('Synthetic Terrain (25° slope with channel)')
ax.set_xlabel('Column (cells)')
ax.set_ylabel('Row (cells)')
plt.colorbar(im, ax=ax, label='Elevation (m)')
plt.show()

## 2. Configuring Flow Parameters

Set up the physical properties of the debris flow material.

In [ ]:
# Create flow parameters for a debris flow
params = pydebflow.FlowParameters(
    solid_density=2500.0,      # kg/m³ (rock/debris)
    fluid_density=1100.0,      # kg/m³ (muddy water)
    basal_friction_angle=22.0, # degrees
    voellmy_mu=0.15,           # Coulomb friction coefficient
    voellmy_xi=500.0           # Turbulent friction (m/s²)
)

print("Flow Parameters:")
print(f"  Solid density: {params.solid_density} kg/m³")
print(f"  Fluid density: {params.fluid_density} kg/m³")
print(f"  Basal friction: {params.basal_friction_angle}°")
print(f"  Voellmy μ: {params.voellmy_mu}")
print(f"  Voellmy ξ: {params.voellmy_xi} m/s²")

In [ ]:
# Create the two-phase flow model
model = pydebflow.TwoPhaseFlowModel(params)

# Configure the solver
solver_config = pydebflow.SolverConfig(
    cfl_number=0.4,
    max_timestep=0.5,
    flux_limiter="minmod"
)

solver = pydebflow.NOCTVDSolver(terrain, model, solver_config)
print("Solver configured successfully!")

## 3. Setting Up Initial Conditions

Create a release zone (initial debris mass) on the terrain.

In [ ]:
# Create initial flow state (all zeros)
state = pydebflow.FlowState.zeros((terrain.rows, terrain.cols))

# Create a circular release zone near the top of the slope
release = terrain.create_release_zone(
    center_i=15,    # Row (near top)
    center_j=30,    # Column (center)
    radius=8,       # Radius in cells
    height=5.0      # Maximum height in meters
)

# Split into solid (70%) and fluid (30%) phases
state.h_solid = release * 0.7
state.h_fluid = release * 0.3

# Calculate initial volume
initial_volume = (state.h_solid.sum() + state.h_fluid.sum()) * terrain.cell_size**2
print(f"Initial release volume: {initial_volume:,.0f} m³")
print(f"Maximum release height: {release.max():.1f} m")

In [ ]:
# Visualize the initial setup
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Terrain
ax = axes[0]
im = ax.imshow(terrain.elevation, cmap='terrain', origin='upper')
ax.set_title('Terrain Elevation')
plt.colorbar(im, ax=ax, label='Elevation (m)')

# Release zone overlaid on terrain
ax = axes[1]
ax.imshow(terrain.elevation, cmap='terrain', origin='upper', alpha=0.7)
im = ax.imshow(np.ma.masked_where(release < 0.01, release), 
               cmap='Reds', origin='upper', alpha=0.8)
ax.set_title('Release Zone on Terrain')
plt.colorbar(im, ax=ax, label='Release Height (m)')

plt.tight_layout()
plt.show()

## 4. Running the Simulation

Execute the debris flow simulation.

In [ ]:
# Run simulation for 30 seconds
print("Running simulation...")

def progress_callback(progress, time, step):
    if step % 100 == 0:
        print(f"  Progress: {progress*100:.1f}%, t={time:.1f}s, steps={step}")

outputs = solver.run_simulation(
    state,
    t_end=30.0,           # Simulation duration (seconds)
    output_interval=2.0,  # Save snapshot every 2 seconds
    progress_callback=progress_callback
)

print(f"\nSimulation complete!")
print(f"Total snapshots: {len(outputs)}")

## 5. Analyzing Results

Process simulation outputs.

In [ ]:
# Extract maximum values
max_height = np.zeros((terrain.rows, terrain.cols))
max_velocity = np.zeros((terrain.rows, terrain.cols))

for t, s in outputs:
    h_total = s.h_solid + s.h_fluid
    speed = np.sqrt(s.u_solid**2 + s.v_solid**2)
    
    max_height = np.maximum(max_height, h_total)
    max_velocity = np.maximum(max_velocity, speed)

# Get final state
t_final, final_state = outputs[-1]
final_height = final_state.h_solid + final_state.h_fluid
final_volume = final_height.sum() * terrain.cell_size**2

print("Simulation Results:")
print(f"  Maximum flow height: {max_height.max():.2f} m")
print(f"  Maximum velocity: {max_velocity.max():.2f} m/s")
print(f"  Final volume: {final_volume:,.0f} m³")
print(f"  Affected area: {np.sum(max_height > 0.1) * terrain.cell_size**2:,.0f} m²")

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Maximum flow height
ax = axes[0]
im = ax.imshow(np.ma.masked_where(max_height < 0.01, max_height),
               cmap='Blues', origin='upper')
ax.set_title('Maximum Flow Height')
plt.colorbar(im, ax=ax, label='Height (m)')

# Maximum velocity
ax = axes[1]
im = ax.imshow(np.ma.masked_where(max_velocity < 0.01, max_velocity),
               cmap='Reds', origin='upper')
ax.set_title('Maximum Velocity')
plt.colorbar(im, ax=ax, label='Velocity (m/s)')

# Final deposit
ax = axes[2]
ax.imshow(terrain.elevation, cmap='terrain', origin='upper', alpha=0.5)
im = ax.imshow(np.ma.masked_where(final_height < 0.01, final_height),
               cmap='YlOrBr', origin='upper', alpha=0.9)
ax.set_title('Final Deposit on Terrain')
plt.colorbar(im, ax=ax, label='Deposit Height (m)')

plt.tight_layout()
plt.show()

## 6. Time-Series Animation

Visualize the flow evolution over time.

In [ ]:
# Create a time-series plot
n_snapshots = min(9, len(outputs))
indices = np.linspace(0, len(outputs)-1, n_snapshots, dtype=int)

fig, axes = plt.subplots(3, 3, figsize=(14, 12))

for i, idx in enumerate(indices):
    t, s = outputs[idx]
    h_total = s.h_solid + s.h_fluid
    
    ax = axes.flat[i]
    ax.imshow(terrain.elevation, cmap='terrain', origin='upper', alpha=0.5)
    im = ax.imshow(np.ma.masked_where(h_total < 0.01, h_total),
                   cmap='Blues', origin='upper', alpha=0.9,
                   vmin=0, vmax=max_height.max())
    ax.set_title(f't = {t:.1f} s')
    ax.set_xticks([])
    ax.set_yticks([])

fig.suptitle('Debris Flow Evolution Over Time', fontsize=14)
fig.colorbar(im, ax=axes.ravel().tolist(), label='Flow Height (m)', shrink=0.6)
plt.tight_layout()
plt.show()

## 7. Using Rheology Models

PyDebFlow supports different rheology models for various flow types.

In [ ]:
# Voellmy model (standard for avalanches)
voellmy = pydebflow.Voellmy(mu=0.15, xi=500.0)
print(f"Voellmy: μ={voellmy.mu}, ξ={voellmy.xi}")

# Using presets
snow_dry = pydebflow.Voellmy.from_preset('snow_dry')
debris_flow = pydebflow.Voellmy.from_preset('debris_flow')

print(f"Snow (dry) preset: μ={snow_dry.mu}, ξ={snow_dry.xi}")
print(f"Debris flow preset: μ={debris_flow.mu}, ξ={debris_flow.xi}")

In [ ]:
# Mohr-Coulomb model (simple friction)
mohr_coulomb = pydebflow.MohrCoulomb(friction_angle=30.0)
print(f"Mohr-Coulomb: friction angle = {mohr_coulomb.friction_angle}°")

## 8. Exporting Results

Save simulation results for further analysis.

In [ ]:
# Create simulation results object
results = pydebflow.SimulationResults(
    times=[t for t, _ in outputs],
    max_flow_height=max_height,
    max_velocity=max_velocity,
    max_pressure=np.zeros_like(max_height),  # placeholder
    final_h_solid=final_state.h_solid,
    final_h_fluid=final_state.h_fluid,
    final_u=final_state.u_solid,
    final_v=final_state.v_solid
)

print(f"Results object created with {len(results.times)} timesteps")
print(f"Total simulation time: {results.times[-1]:.1f} s")

## Summary

This notebook demonstrated:

1. **Creating terrain** - Synthetic slopes or loading DEMs
2. **Configuring flow parameters** - Material properties and rheology
3. **Setting up release zones** - Initial debris mass
4. **Running simulations** - NOC-TVD solver execution
5. **Analyzing results** - Max height, velocity, affected area
6. **Visualizing** - Time-series evolution and final deposits
7. **Rheology models** - Voellmy, Mohr-Coulomb presets
8. **Exporting** - Saving results for further analysis

For more information, see [PyDebFlow on GitHub](https://github.com/ankitdutta428/PyDebFlow).